## 朴素贝叶斯


> 使用贝叶斯公式对离散数据进行分类

### 贝叶斯公式：
对于事件$B_i$,在$A$ 发生的条件下发生的概率为：
$$ P(B_i | A) =\frac { P(B_i ) P ( A|B_i)}{ P(A)}$$

对于训练数据集，一组数据包括多个属性取值的向量$X$ 和目标类别$c$ 。

当我们取得一个新的属性向量$X$后，要判断其所属的类别，可以这样:

> 针对每个可能的类别$c_i$ 计算 $P(c_i|X)$,取拥有最大$P(c_i|X)$的$c_i$ 作为$X$的类别。

这就是朴素贝叶斯算法的核心思想

### 朴素贝叶斯算法

计算$P(c_i|X)$ 可根据贝叶斯公式:
$$ P(c_i | X) =\frac { P(c_i ) P ( X|c_i)}{ P(X)}$$
进行求解。由于$P(x) $ 对于每一个$c_i$ 都一样，所以要求最大的$P(c_i|X)$ 等价于求最大的$P(c_i ) P ( X|c_i)$

$P(c_i )$ 可使用训练数据的$c_i$的频率进行代替。

下面讨论$ P ( X|c_i)$的计算过程：

如果直接在训练集里面直接在$c_i$取$X$出现的频率当作概率误差较大，有可能该属性取值组合出现次数很少或者为零，但是不能说明这个组合为这个类别的几率少。为了更好的计算$ P ( X|c_i)$，我们引入**独立性假设** 
#### 独立性假设
> 假设各个属性取值的相互独立的

这个假设对于实际数据几乎是达不到的，这也是算法的误差来源之一。可以使用一些数据预处理的手段降低这个误差，我们先认为由于这个引起的误差足够小。

当数据满足这个假设时，$$ P ( X|c_i) = \prod _{j=1}^{n} P(x_j|c_i)$$ $x_j$为$X$各个属性的取值。

#### 计算公式
整理前面的公式我们可以得到：

对每个可能的类别$c_i$ 的$P(c_i|X) $ 有：

$$P(c_i|X) \propto P(c_i ) P ( X|c_i) =P(c_i ) \prod _{j=1}^{n} P(x_j|c_i) $$

公式的右部我们可根据训练数据得到，这就是朴素贝叶斯算法。

代码框架如下：

In [41]:
import pandas as pd
import numpy as np
def  bayesian(dataset :pd.DataFrame ,X :pd.Series):
    allclass = getAllClass(dataset)
    inittable(dataset)
    theclass=None
    theP=-1
    for c in allclass:
        cP=getP(X,c)
        if(cP>theP):
            theclass=c
            theP=cP
    return theclass;

In [42]:
def getAllClass(df):
    return df[df.columns[-1]].unique();

In [48]:
theP=[]
thecp={}
def getPre(df,i,xi,ci):
    subdf =df[df[df.columns[-1]]==ci]
    c = subdf.shape[0]
    cc = subdf [ subdf[subdf.columns[i]] == xi] . shape[0]
    return cc/c
def inittable(df):
    for i in range(df.shape[1]-1): #属性数量
        theP.append({})
        d=theP[i]
        for xi in (df[df.columns[i]].unique()):
            dd={}
            d[xi]=dd
            for ci in df[df.columns[-1]].unique():
                dd[ci]=getPre(df,i,xi,ci);
    tc=df.shape[0];
    for ci in df[df.columns[-1]].unique():
        subdf =df[df[df.columns[-1]]==ci]
        c = subdf.shape[0]
        thecp[ci]=c/tc

def getP(X,c):
    res=1;
    res*=thecp[c]
    for i in range(X.count()):
        res*=theP[i][X[i]][c]
    return res

In [49]:
outlook=[1,1,2,3,3,3,2,1,1,3,1,2,2,3]
temp=[1,1,1,2,3,3,3,2,3,2,2,2,1,2]
hum=[1,1,1,1,2,2,2,1,2,2,2,1,2,1]
windy=[0,1,0,0,0,1,1,0,0,0,1,1,0,0]
play=[0,0,1,1,1,0,1,0,1,1,1,1,1,0]
DataSet = list(zip(outlook,temp,hum,windy,play))
playdf = pd.DataFrame(data = DataSet, columns=["outlook","temp","hum","windy","play"])
playdf

,outlook,temp,hum,windy,play
0,1,1,1,0,0
1,1,1,1,1,0
2,2,1,1,0,1
3,3,2,1,0,1
4,3,3,2,0,1
5,3,3,2,1,0
6,2,3,2,1,1
7,1,2,1,0,0
8,1,3,2,0,1
9,3,2,2,0,1


In [50]:
playdf [ playdf["play" ] ==0     ]

,outlook,temp,hum,windy,play
0,1,1,1,0,0
1,1,1,1,1,0
5,3,3,2,1,0
7,1,2,1,0,0
13,3,2,1,0,0


In [51]:
bayesian(playdf,pd.Series([1,1,1,0]))

0

In [52]:
bayesian(playdf,pd.Series([2,2,1,1]))

1

In [53]:
theP

[{1: {0: 0.6, 1: 0.2222222222222222},
  2: {0: 0.0, 1: 0.4444444444444444},
  3: {0: 0.4, 1: 0.3333333333333333}},
 {1: {0: 0.4, 1: 0.2222222222222222},
  2: {0: 0.4, 1: 0.4444444444444444},
  3: {0: 0.2, 1: 0.3333333333333333}},
 {1: {0: 0.8, 1: 0.3333333333333333}, 2: {0: 0.2, 1: 0.6666666666666666}},
 {0: {0: 0.6, 1: 0.6666666666666666}, 1: {0: 0.4, 1: 0.3333333333333333}},
 {},
 {},
 {},
 {}]